In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [3]:
#Read all data.

main_puf = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_TRAUMA.csv", index_col = 'inc_key', low_memory = False)
preexisting_conditions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_PREEXISTINGCONDITIONS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
hospital_events = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_HOSPITALEVENTS_PIVOT.csv", index_col = 'Inc_Key', low_memory = False)
ais = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_AISDIAGNOSIS_PIVOT.csv", index_col = 'inc_key', low_memory = False)
ecode = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_ECODE.csv", index_col = 'inc_key', low_memory = False)
interventions = pd.read_csv("/content/drive/MyDrive/NTDB-PUFs/NTDB_2020/PUF_INTERVENTIONS.csv", index_col = 'Inc_Key', low_memory = False)

In [4]:
data = pd.concat([main_puf, preexisting_conditions, hospital_events, ais, ecode, interventions], axis=1)
data.columns = map(str.upper, data.columns)
data.shape

(1135018, 352)

#Inclusion and Exclusion Criteria

In [5]:
#Identify the patient population.

data = data[(data['AISSEVERITY1'] == 2) | (data['AISSEVERITY1'] == 3) | (data['AISSEVERITY1'] == 4) | (data['AISSEVERITY1'] == 5)]
data = data[(data['VERIFICATIONLEVEL'] == 1) | (data['VERIFICATIONLEVEL'] == 2)]

included = data.shape[0]

print('Number of patients included: ', included)

Number of patients included:  174376


In [6]:
#Exclude pediatric patients.

before = data.shape[0]
data = data[(data['AGEYEARS'] >= 18)]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  174376
Number of patients after exclusion:  152512
Number of patients excluded with this criteria:  21864


In [7]:
#Exclude AIS injury severity score ≥ 3 to neck.

before = data.shape[0]
data = data[data['AISSEVERITY3'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  152512
Number of patients after exclusion:  150592
Number of patients excluded with this criteria:  1920


In [8]:
#Exclude AIS injury severity score ≥ 3 to thorax.

before = data.shape[0]
data = data[data['AISSEVERITY4'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  150592
Number of patients after exclusion:  128040
Number of patients excluded with this criteria:  22552


In [9]:
#Exclude AIS injury severity score ≥ 3 to abdomen.

before = data.shape[0]
data = data[data['AISSEVERITY5'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  128040
Number of patients after exclusion:  126224
Number of patients excluded with this criteria:  1816


In [10]:
#Exclude AIS injury severity score ≥ 3 to spine.

before = data.shape[0]
data = data[data['AISSEVERITY6'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  126224
Number of patients after exclusion:  122358
Number of patients excluded with this criteria:  3866


In [11]:
#Exclude AIS injury severity score ≥ 3 to upper extremity.

before = data.shape[0]
data = data[data['AISSEVERITY7'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  122358
Number of patients after exclusion:  121685
Number of patients excluded with this criteria:  673


In [12]:
#Exclude AIS injury severity score ≥ 3 to lower extremity.

before = data.shape[0]
data = data[data['AISSEVERITY8'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  121685
Number of patients after exclusion:  116683
Number of patients excluded with this criteria:  5002


In [13]:
#Exclude AIS injury severity score ≥ 3 to unspecified body regions.

before = data.shape[0]
data = data[data['AISSEVERITY9'] < 3]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  116683
Number of patients after exclusion:  116651
Number of patients excluded with this criteria:  32


In [14]:
#Exclude patients dead on arrival (SBP=0).

before = data.shape[0]
data = data[data['SBP'] != 0]
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  116651
Number of patients after exclusion:  115960
Number of patients excluded with this criteria:  691


In [15]:
#Exclude patients with advanced directive limiting care.

before = data.shape[0]
data = data[data['CC_ADLC'] != 'Yes']
after = data.shape[0]
excluded = before - after

print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  115960
Number of patients after exclusion:  111100
Number of patients excluded with this criteria:  4860


In [16]:
#Drop patients with invalid GCS.

before = data.shape[0]
data = data[data['PMGCSQ_VALID'] == 1]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  111100
Number of patients after exclusion:  93332
Number of patients excluded with this criteria:  17768


In [17]:
#Drop patients with unknown GCS.

before = data.shape[0]
data = data[data['TOTALGCS'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  93332
Number of patients after exclusion:  90419
Number of patients excluded with this criteria:  2913


In [18]:
#Drop patients with unknown GCS-Motor.

before = data.shape[0]
data = data[data['GCSMOTOR'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  90419
Number of patients after exclusion:  90320
Number of patients excluded with this criteria:  99


In [19]:
#Drop patients with unknown GCS-Verbal.

before = data.shape[0]
data = data[data['GCSVERBAL'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  90320
Number of patients after exclusion:  90315
Number of patients excluded with this criteria:  5


In [20]:
#Drop patients with unknown GCS-Eye.

before = data.shape[0]
data = data[data['GCSEYE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  90315
Number of patients after exclusion:  90314
Number of patients excluded with this criteria:  1


In [21]:
#Drop patients with unknown pupillary response.

before = data.shape[0]
data = data[data['TBIPUPILLARYRESPONSE'].notna()]
after = data.shape[0]

excluded = before - after
print('Number of patients before exclusion: ', before)
print('Number of patients after exclusion: ', after)
print('Number of patients excluded with this criteria: ', excluded)

Number of patients before exclusion:  90314
Number of patients after exclusion:  87695
Number of patients excluded with this criteria:  2619


In [22]:
#Save data.

data.to_csv('/content/drive/MyDrive/TQP-MOST/2020.csv')